# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. 

In [2]:
# Your code here:
data = pd.read_csv("Current_Employee_Names__Salaries__and_Position_Titles.csv")
display(data.shape)
display(data.dtypes)

(33702, 8)

Name                  object
Job Titles            object
Department            object
Full or Part-Time     object
Salary or Hourly      object
Typical Hours        float64
Annual Salary        float64
Hourly Rate          float64
dtype: object

Examine the `salaries` dataset using the `head` function below.

and put snakecase while we're at it.

In [3]:
# Your code here:
cols=[]
for i in data.columns:
    cols.append(i.lower())
data.columns = cols
data.columns = data.columns.str.replace(' ', '_')
display(data.head())


,name,job_titles,department,full_or_part-time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"ADRIANO, RACQUEL ANNE",PARAMEDIC,FIRE,F,Salary,NaN,72510.0,NaN
1,"AFFANEH, MAHIR A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
2,"AFOENYI, PHYLLIS",TITLE V PROGRAM TRAINEE I,FAMILY & SUPPORT,P,Hourly,20.0,NaN,13.0
3,"AGAR, BULENT B",DEPUTY COMMISSIONER,WATER MGMNT,F,Salary,NaN,132972.0,NaN
4,"AGEE, LAWRENCE",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
data.isna().sum()

name                     0
job_titles               0
department               0
full_or_part-time        0
salary_or_hourly         0
typical_hours        25528
annual_salary         8174
hourly_rate          25528
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
data["salary_or_hourly"].value_counts()


Salary    25528
Hourly     8174
Name: salary_or_hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:
data.pivot_table(index = "department", values = "name", aggfunc = {"name":"count"})


,name
department,
ADMIN HEARNG,38
ANIMAL CONTRL,75
AVIATION,1856
BOARD OF ELECTION,111
BOARD OF ETHICS,8
BUDGET & MGMT,44
BUILDINGS,258
BUSINESS AFFAIRS,167
CITY CLERK,88


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
data_hourly = data.dropna(subset = "hourly_rate")
data_hourly = data_hourly.drop("annual_salary", axis = 1)
display(data_hourly.shape)
display(data_hourly.isna().sum())

(8174, 7)

name                 0
job_titles           0
department           0
full_or_part-time    0
salary_or_hourly     0
typical_hours        0
hourly_rate          0
dtype: int64

In [25]:
# Your code here:
# so the H0 would be hourly salary == 30.0
# means the H1 would be hourly salary <> 30.0
import scipy.stats
confidence_level1 = 0.95
degrees_freedom1 = (data_hourly.shape[0]) - 1
sample_mean1 = data_hourly["hourly_rate"].mean()
sample_standard_error1 = scipy.stats.sem(data_hourly["hourly_rate"])
confidence_interval1 = scipy.stats.t.interval(confidence_level1, 
                                             degrees_freedom1, 
                                             sample_mean1, 
                                             sample_standard_error1)

In [27]:
print( 'confidence interval:', confidence_interval1, '.' )

confidence interval: (34.29839539204317, 34.874504045196865) .


In [29]:
from scipy.stats import ttest_1samp
stat1, pval1 = ttest_1samp(data_hourly["hourly_rate"], 30)

In [30]:
print('stat\t\t\t\t:', stat1)
print('pvalue for the two-tailed test\t:', pval1)

stat				: 31.211500329216044
pvalue for the two-tailed test	: 3.825475847673794e-202


In [ ]:
# the p-value is 0.00000... a lot smaller than 0.5.
# means we reject the null hypothesis H0.
# means the hourly rate is not equal 30.0$/hr

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [23]:
# Your code here:
data_salary = data.dropna(subset = "annual_salary")
data_salary = data_salary.drop(["hourly_rate","typical_hours"], axis = 1)
data_police = data_salary[data_salary["department"] == "POLICE"]
display(data_police.shape)
display(data_police.isna().sum())


(13824, 6)

name                 0
job_titles           0
department           0
full_or_part-time    0
salary_or_hourly     0
annual_salary        0
dtype: int64

In [33]:
# H0: annual salary <= 86000
# H1: annual salary > 86000
confidence_level2 = 0.95
degrees_freedom2 = (data_police.shape[0]) - 1
sample_mean2 = data_police["annual_salary"].mean()
sample_standard_error2 = scipy.stats.sem(data_police["annual_salary"])
confidence_interval2 = scipy.stats.t.interval(confidence_level2, 
                                             degrees_freedom2, 
                                             sample_mean2, 
                                             sample_standard_error2)

In [32]:
print( 'confidence interval:', confidence_interval2, '.' )

confidence interval: (88534.1182885883, 89134.11955863392) .


In [36]:
stat2, pval2 = ttest_1samp(data_police["annual_salary"], 86000)
print('stat\t\t\t\t:', stat2)
print('pvalue for the two-tailed test\t:', pval2/2)

stat				: 18.51748561981861
pvalue for the two-tailed test	: 6.119566952121482e-76


In [ ]:
# we reject the "equal" part because the p-value is way lower than 0.05
# stat is positive, that means the symbol should be "greater than"
# therefore we can reject the null hypothesis

Using the `crosstab` function, find the department that has the most hourly workers. 

In [47]:
# Your code here:
data_crosstab = pd.crosstab(data_hourly.department, data_hourly["salary_or_hourly"])
data_crosstab.sort_values(by = "Hourly", ascending = 0).head(1)

salary_or_hourly,Hourly
department,
STREETS & SAN,1846


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [50]:
data_ss = data_hourly[data_hourly["department"] == "STREETS & SAN"]

In [51]:
# Your code here:
# h0: hourly rate >= 35.0
# h1: hourly rate < 35.0
confidence_level3 = 0.95
degrees_freedom3 = (data_ss.shape[0]) - 1
sample_mean3 = data_ss["hourly_rate"].mean()
sample_standard_error3 = scipy.stats.sem(data_ss["hourly_rate"])
confidence_interval3 = scipy.stats.t.interval(confidence_level3, 
                                             degrees_freedom3, 
                                             sample_mean3, 
                                             sample_standard_error3)


In [52]:
print( 'confidence interval:', confidence_interval3, '.' )

confidence interval: (35.28150811892106, 35.82308559722194) .


In [53]:
stat3, pval3 = ttest_1samp(data_ss["hourly_rate"], 35)
print('stat\t\t\t\t:', stat3)
print('pvalue for the two-tailed test\t:', pval3)

stat				: 4.000138803968834
pvalue for the two-tailed test	: 6.580474443773579e-05


In [ ]:
# the pvalue is so low, we reject the equal part where hourly rate = 35.0
# but the stat is positive, so we dont reject the part where hourly rate > 35.0
# and we see that the confidence interval is between 35.28 and 35.82.
# so there is 95% certainty that the mean of the hourly rate lies in that interval (is this correct?)

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [54]:
# Your code here:
confidence_level1 = 0.95
degrees_freedom1 = (data_hourly.shape[0]) - 1
sample_mean1 = data_hourly["hourly_rate"].mean()
sample_standard_error1 = scipy.stats.sem(data_hourly["hourly_rate"])
confidence_interval1 = scipy.stats.t.interval(confidence_level1, 
                                             degrees_freedom1, 
                                             sample_mean1, 
                                             sample_standard_error1)
print( 'confidence interval:', confidence_interval1, '.' )

confidence interval: (34.29839539204317, 34.874504045196865) .


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [55]:
# Your code here:
onfidence_level2 = 0.95
degrees_freedom2 = (data_police.shape[0]) - 1
sample_mean2 = data_police["annual_salary"].mean()
sample_standard_error2 = scipy.stats.sem(data_police["annual_salary"])
confidence_interval2 = scipy.stats.t.interval(confidence_level2, 
                                             degrees_freedom2, 
                                             sample_mean2, 
                                             sample_standard_error2)
print( 'confidence interval:', confidence_interval2, '.' )

confidence interval: (88534.1182885883, 89134.11955863392) .


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [57]:
pip install statsmodels

  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.2
    Uninstalling numpy-1.22.2:
      Successfully uninstalled numpy-1.22.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\Python 3.10\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'd:\Python 3.10\python.exe -m pip install --upgrade pip' command.


In [58]:
# Your code here:
# h0: number of hourly worker = 25% of total worker
# h1: number of hourly worker <> 25 of total worker
from statsmodels.stats.proportion import proportions_ztest
data_prop = data.dropna(subset = "hourly_rate")
data_prop = data_prop.drop(["annual_salary"], axis = 1)
display(data_prop.shape)
display(data_prop.isna().sum())


ModuleNotFoundError: No module named 'statsmodels'

In [59]:
# hmmm....